# Importing libraries

In [ ]:
import sys, os, re, csv, subprocess, operator
import pandas as pd
from urllib.request import urlopen
import urllib.request
from pprint import pprint
from bs4 import BeautifulSoup

# Defining repository and directories

In [ ]:
userhome = os.path.expanduser("~")
repository = userhome + r"/different-diff/dataset/openjpa/"
analyze_dir = userhome + r'/different-diff/analyze/analyze_openjpa/'

# Load textfile contains bug-ids

In [ ]:
txt_file = open(analyze_dir + "01_code_document/txt_files/candidate_bug_links.txt", "r")
bug_links = txt_file.read().split('\n')
print ("Found " + str(len(bug_links)) + " bug_ids")

In [ ]:
print (bug_links)

# Finding affected versions by bug links

In [ ]:
error_links = []
affected_version = []
for a,b in enumerate(bug_links):
    link = "https://issues.apache.org/jira/browse/" + b
    sys.stdout.write("\r%i " %(a+1) + "Extracting: " + b)
    sys.stdout.flush()
    try:
        page = urllib.request.urlopen(link)
        soup = BeautifulSoup(page, 'html.parser')
        aff_version = soup.find('span', attrs={'id':'versions-val'}).text.replace("\n",'').replace(" ",'').split(",")
        aff_version = sorted(aff_version)
        aff_version.insert(0, b)
        affected_version.append(aff_version)
    except:
        error_links.append(b)

print("\nExtraction has been completed.")

In [ ]:
print (error_links)

In [ ]:
#Repeat the process if there are still some affected versions by bug_ids haven't been captured
errorlinks = []
if error_links != []:
    for c,d in enumerate(error_links):
        link = "https://issues.apache.org/jira/browse/" + d
        sys.stdout.write("\r%i " %(c+1) + "Extracting: " + d)
        sys.stdout.flush()
        try:
            page = urllib.request.urlopen(link)
            soup = BeautifulSoup(page, 'html.parser')
            types = soup.find('span', attrs={'id':'versions-val'}).text.replace("\n",'').replace(" ",'').split(",")
            types = sorted(types)
            types.insert(0, d)
            affected_version.append(types)
        except:
            errorlinks.append(b)

print ("\nExtraction is complete")

In [ ]:
print (errorlinks)

In [ ]:
affected_version.sort()
affected_version

In [ ]:
#Finding the earliest version affected by the bug ids
earliest_version = []
for num, affver in enumerate(affected_version):
    earliest_version.append(affver[:2])

earliest_version.sort()
for early in earliest_version:
    print (early)

# Defining the function for git command

In [ ]:
def execute_command(cmd, work_dir):
    #Executes a shell command in a subprocess, waiting until it has completed.
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

# Finding the versions related with earliest version

In [ ]:
related_version = []
for n, item in enumerate(earliest_version):
    if item[1] != "None":
        git_cmd = 'git tag -l "*' + item[1] + '*"'
        temp = str(execute_command(git_cmd, repository)).replace("b'",'').replace("(",'').replace(")",'').split("\\n")
        del temp[len(temp)-1]
        if temp == []:
            temp = [item[1]]
    else:
        temp = ['None']

    temp.insert(0, item[0])
    related_version.append(temp)

for xx in related_version:
    print (xx)

# Finding the date release for affected version

In [ ]:
related_version[1][1]

In [ ]:
date_release = []
for n, item in enumerate(related_version):
    sys.stdout.write("\rFinding datetime for version {}: {}".format(n+1, item[0]))
    sys.stdout.flush()
    if item[1] != "None":
        for m in range(1, len(item)):
            git_cmd = "git log -1 --format=%ai " + item[m]
            temp = str(execute_command(git_cmd, repository)).replace("b'",'').replace("(",'').replace(")",'').split("\\n")
            del temp[len(temp)-1]
            temp = temp[0].split(" ")
            if temp[0] != "',":
                temp.insert(0,item[0])
                temp.insert(1,item[m])
                date_release.append(temp)
                date_release = sorted(date_release, key=operator.itemgetter(0, 2))
    """else:
        date_release.append(item)"""

date_release = sorted(date_release, key=operator.itemgetter(0), reverse=True) 

print("\n")
for jj in date_release:
    print (jj)

In [ ]:
#save in CSV file
with open(analyze_dir + '05_finding_versions/01_affected_version/affected_version.csv','w') as csvfile:
    writers = csv.writer(csvfile)
    writers.writerow(['bug_id','earliest_affected_version','date_release','time_release','tz'])
    for item in date_release:
        writers.writerow(item)

In [ ]:
df = pd.read_csv(analyze_dir + '05_finding_versions/01_affected_version/affected_version.csv')
df

In [ ]:
earliest_vers = df.groupby('bug_id', as_index=False).first()
earliest_vers = earliest_vers.sort_values(['date_release', 'time_release', 'earliest_affected_version'], ascending=True)
earliest_vers.to_csv(analyze_dir + '05_finding_versions/01_affected_version/earliest_version.csv')
earliest_vers

# Finding fix versions of bug links

In [ ]:
errors = []
fixed_version = []
for a,b in enumerate(bug_links):
    link = "https://issues.apache.org/jira/browse/" + b
    sys.stdout.write("\r%i " % (a+1) + "Extracting: " + b)
    sys.stdout.flush()
    try:
        page = urllib.request.urlopen(link)
        soup = BeautifulSoup(page, 'html.parser')
        fix_version = soup.find('span', attrs={'id':'fixfor-val'}).text.replace("\n",'').replace(" ",'').split(",")
        fix_version = sorted(fix_version)
        fix_version.insert(0, b)
        fixed_version.append(fix_version)
    except:
        errors.append(b)

print("\nExtraction has been completed.")

In [ ]:
fixed_versions = fixed_version
errors

In [ ]:
error = []
for a,b in enumerate(errors):
    link = "https://issues.apache.org/jira/browse/" + b
    sys.stdout.write("\r%i " % (a+1) + "Extracting: " + b)
    sys.stdout.flush()
    try:
        page = urllib.request.urlopen(link)
        soup = BeautifulSoup(page, 'html.parser')
        fix_version = soup.find('span', attrs={'id':'fixfor-val'}).text.replace("\n",'').replace(" ",'').split(",")
        fix_version = sorted(fix_version)
        fix_version.insert(0, b)
        fixed_versions.append(fix_version)
    except:
        error.append(b)

print("\nExtraction has been completed.")

In [ ]:
error

In [ ]:
fixed_versions.sort()
for itemfix in fixed_versions:
    print (itemfix)